In [1]:
from dotenv import load_dotenv
import os
import json
import re

import pdfplumber

from h2ogpte import H2OGPTE

In [2]:
jd_path = '../JDs/'
pdf_files = [file for file in os.listdir(jd_path) if file.endswith(".pdf")]
pdf_files

['Digital BCG - Data Scientist.pdf',
 'Robust Intelligence - Machine Learning Engineer.pdf',
 'Eli Lilly - Data Scientist, Global Report.pdf',
 'OpenAI - Principal Solutions Engineer – Japan.pdf',
 'Money Forward - AI Soultions Engineer.pdf',
 'AWS - Sr. Generative AI Specialist, Generative AI Innovation Center.pdf',
 'Senior Data Scientist @ Dataiku.pdf']

In [3]:
load_dotenv(dotenv_path='.env')
heogpte_key = os.getenv('h2oGpteKey_AgentWorkflow')
if heogpte_key: print('key is set.')

key is set.


In [10]:
client = H2OGPTE(
    address='https://h2ogpte.dev.h2o.ai',
    api_key=heogpte_key,   # Collection Key
)

chat_session_id = client.create_chat_session_on_default_collection()

client, chat_session_id

(<h2ogpte.h2ogpte.H2OGPTE at 0x10c916320>,
 '6df0140f-793f-4c4e-b08a-0c20540ab9fb')

In [11]:
# 利用可能なLLM
client.get_llm_names()

['meta-llama/Meta-Llama-3.1-8B-Instruct',
 'h2oai/h2o-danube3-4b-chat',
 'meta-llama/Llama-3.3-70B-Instruct',
 'Qwen/Qwen2.5-VL-72B-Instruct',
 'h2oai/h2ovl-mississippi-2b',
 'mistralai/Pixtral-12B-2409',
 'mistralai/Mixtral-8x7B-Instruct-v0.1',
 'meta-llama/Meta-Llama-3.1-405B-Instruct',
 'meta-llama/Meta-Llama-3.1-70B-Instruct',
 'mistralai/Mistral-7B-Instruct-v0.3',
 'meta-llama/Llama-3.2-11B-Vision-Instruct-Turbo',
 'mistral-tiny',
 'mistral-small-latest',
 'gemini-1.5-pro-latest',
 'gemini-2.0-flash',
 'deepseek-ai/DeepSeek-V3-together',
 'claude-3-5-sonnet-20241022-bedrock',
 'claude-3-5-haiku-20241022',
 'claude-3-5-sonnet-20241022',
 'claude-3-7-sonnet-20250219',
 'claude-3-7-sonnet-20250219-reasoning',
 'gpt-4o',
 'gpt-4o-mini',
 'Qwen/QwQ-32B',
 'deepseek-ai/DeepSeek-R1-together',
 'gemini-2.0-flash-thinking-exp-01-21']

In [12]:
def llm_call(prompt: str, system_prompt: str = "あなたは優秀なAIアシスタントです。指示に的確に従います。", model='gpt-4o-mini') -> str:
    with client.connect(chat_session_id) as session:
        reply = session.query(
            message=prompt,
            #system_prompt=system_prompt,
            llm=model,
            rag_config = {"rag_type": "llm_only"},
            timeout=60,
        )
        return reply.content

def extract_xml(text: str, tag: str) -> str:
    match = re.search(f'<{tag}>(.*?)</{tag}>', text, re.DOTALL)
    return match.group(1) if match else ""

In [13]:
def generate(prompt: str, task: str, context: str = "") -> tuple[str, str]:
    """Generate and improve a solution based on feedback."""
    full_prompt = f"{prompt}\n{context}\nTask: {task}" if context else f"{prompt}\nTask: {task}"
    print('generate full_prompt >>>>>>>>>>>>>>>>>', full_prompt, '<<<<<<<<<<<<<<<<<<< generate full_prompt')
    response = llm_call(full_prompt)
    thoughts = extract_xml(response, "thoughts")
    result = extract_xml(response, "response")
    
    print("\n=== GENERATION START ===")
    print(f"Thoughts:\n{thoughts}\n")
    print(f"Generated:\n{result}")
    print("=== GENERATION END ===\n")
    
    return thoughts, result

def evaluate(prompt: str, content: str, task: str) -> tuple[str, str]:
    """Evaluate if a solution meets requirements."""
    full_prompt = f"{prompt}\nOriginal task: {task}\nContent to evaluate: {content}"
    print('evaluate full_prompt >>>>>>>>>>>>>>>>>', full_prompt, '<<<<<<<<<<<<<<<<<<< evaluate full_prompt')
    response = llm_call(full_prompt)
    evaluation = extract_xml(response, "evaluation")
    feedback = extract_xml(response, "feedback")
    
    print("=== EVALUATION START ===")
    print(f"Status: {evaluation}")
    print(f"Feedback: {feedback}")
    print("=== EVALUATION END ===\n")
    
    return evaluation, feedback

def loop(task: str, evaluator_prompt: str, generator_prompt: str) -> tuple[str, list[dict]]:
    """Keep generating and evaluating until requirements are met."""
    memory = []
    chain_of_thought = []
    
    thoughts, result = generate(generator_prompt, task)
    memory.append(result)
    chain_of_thought.append({"thoughts": thoughts, "result": result})
    
    while True:
        evaluation, feedback = evaluate(evaluator_prompt, result, task)
        if evaluation == "PASS":
            return result, chain_of_thought
            
        context = "\n".join([
            "Previous attempts:",
            *[f"- {m}" for m in memory],
            f"\nFeedback: {feedback}"
        ])
        
        thoughts, result = generate(generator_prompt, task, context)
        memory.append(result)
        chain_of_thought.append({"thoughts": thoughts, "result": result})

In [14]:
evaluator_prompt = """
与えられたテキストを以下2つの基準で判定してください。
1. 「会社名」「ポジション名」「必要な学歴」「必要な経験年数」「必要な言語スキル」「必要なテクニカルスキル」の6つ全ての項目が抽出されているか
2.  抽出された情報はJSON形式となっているか

タスクの解決を試みることなく、評価のみを行ってください。
すべての基準が満たされ、改善のための提案がない場合は、"PASS"のみを出力してください。
次の形式で簡潔に評価を出力してください。

<evaluation>PASS, NEEDS_IMPROVEMENT, or FAIL</evaluation>
<feedback>
改善が必要な理由
</feedback>
"""

generator_prompt = """
目標は、<user input>に基づいてタスクを完了することです。
フィードバックを受け取った場合は、それを考慮して回答を改善する必要があります。

次の形式で簡潔に回答を出力してください。

<thoughts>
タスクとフィードバックに対する理解、および改善の計画
</thoughts>

<response>
{
    "会社名":"回答",
    "ポジション名:"回答",
    "必要な学歴":"回答",
    "必要な経験年数":"回答",
    "必要な言語スキル":"回答",
    "必要なテクニカルスキル":"回答"
}
</response>
"""

In [15]:
file_path = "../JDs/Money Forward - AI Soultions Engineer.pdf"

with pdfplumber.open(file_path) as pdf:
    text = "\n".join([page.extract_text() for page in pdf.pages if page.extract_text()])

#print(text)

task = """
以下のJob Descriptionから、「会社名」「ポジション名」「必要な学歴」「必要な経験年数」「必要な言語スキル」「必要なテクニカルスキル」の6つを抽出してください。
Job Description内に情報がない場合は、'情報なし'と出力してください。

# Job Description
{}
""".format(text)

#print(task)

reply = loop(task, evaluator_prompt, generator_prompt)
reply

generate full_prompt >>>>>>>>>>>>>>>>> 
目標は、<user input>に基づいてタスクを完了することです。
フィードバックを受け取った場合は、それを考慮して回答を改善する必要があります。

次の形式で簡潔に回答を出力してください。

<thoughts>
タスクとフィードバックに対する理解、および改善の計画
</thoughts>

<response>
{
    "会社名":"回答",
    "ポジション名:"回答",
    "必要な学歴":"回答",
    "必要な経験年数":"回答",
    "必要な言語スキル":"回答",
    "必要なテクニカルスキル":"回答"
}
</response>

Task: 
以下のJob Descriptionから、「会社名」「ポジション名」「必要な学歴」「必要な経験年数」「必要な言語スキル」「必要なテクニカルスキル」の6つを抽出してください。
Job Description内に情報がない場合は、'情報なし'と出力してください。

# Job Description
Main business activities
In order to accelerate the growth of Money Forward's services and products, you will be working on
proposing, verifying and developing function development and problem-solving methods using
AI/machine learning.
You will be responsible for defining issues from the perspective of user business and product
specifications, and for considering solutions using AI/machine learning, as well as implementing and
operating them.
● Interview with internal business and back office departments a

('\n{\n    "会社名": "Money Forward",\n    "ポジション名": "AI/Machine Learning Engineer",\n    "必要な学歴": "情報なし",\n    "必要な経験年数": "情報なし",\n    "必要な言語スキル": "基本的なビジネスレベルの英語スキル（TOEIC 700以上相当）",\n    "必要なテクニカルスキル": "AIサービス、TensorFlow、Keras、Python、AWS、GCP、SQLなど"\n}\n',
 [{'thoughts': '\nJob Descriptionから必要な情報を抽出するタスクを理解しました。会社名、ポジション名、必要な学歴、必要な経験年数、必要な言語スキル、必要なテクニカルスキルの6つの要素を特定する必要があります。Job Descriptionには具体的な学歴や経験年数の情報が含まれていないため、これらは「情報なし」として出力します。言語スキルとテクニカルスキルは明確に記載されているため、それに基づいて回答を作成します。\n',
   'result': '\n{\n    "会社名": "Money Forward",\n    "ポジション名": "AI/Machine Learning Engineer",\n    "必要な学歴": "情報なし",\n    "必要な経験年数": "情報なし",\n    "必要な言語スキル": "基本的なビジネスレベルの英語スキル（TOEIC 700以上相当）",\n    "必要なテクニカルスキル": "AIサービス、TensorFlow、Keras、Python、AWS、GCP、SQLなど"\n}\n'}])

In [16]:
file_path = "../JDs/Senior Data Scientist @ Dataiku.pdf"

with pdfplumber.open(file_path) as pdf:
    text = "\n".join([page.extract_text() for page in pdf.pages if page.extract_text()])

#print(text)

task = """
以下のJob Descriptionから、「会社名」「ポジション名」「必要な学歴」「必要な経験年数」「必要な言語スキル」「必要なテクニカルスキル」の6つを抽出してください。
Job Description内に情報がない場合は、'情報なし'と出力してください。

# Job Description
{}
""".format(text)

#print(task)

reply = loop(task, evaluator_prompt, generator_prompt)
reply

generate full_prompt >>>>>>>>>>>>>>>>> 
目標は、<user input>に基づいてタスクを完了することです。
フィードバックを受け取った場合は、それを考慮して回答を改善する必要があります。

次の形式で簡潔に回答を出力してください。

<thoughts>
タスクとフィードバックに対する理解、および改善の計画
</thoughts>

<response>
{
    "会社名":"回答",
    "ポジション名:"回答",
    "必要な学歴":"回答",
    "必要な経験年数":"回答",
    "必要な言語スキル":"回答",
    "必要なテクニカルスキル":"回答"
}
</response>

Task: 
以下のJob Descriptionから、「会社名」「ポジション名」「必要な学歴」「必要な経験年数」「必要な言語スキル」「必要なテクニカルスキル」の6つを抽出してください。
Job Description内に情報がない場合は、'情報なし'と出力してください。

# Job Description
Client Name: Dataiku
Role: Senior Data Scientist
At Dataiku, we're not just adapting to the AI revolution, we're leading it. Since our beginning in Paris in
2013, we've been pioneering the future of AI with a platform that makes data actionable and accessible.
With over 1,000 teammates across 25 countries and backed by a renowned set of investors, we're the
architects of Everyday AI, enabling data experts and domain experts to work together to build AI into their
daily operations, from advanced analytics t

('\n{\n    "会社名": "Dataiku",\n    "ポジション名": "Senior Data Scientist",\n    "必要な学歴": "情報なし",\n    "必要な経験年数": "8年以上",\n    "必要な言語スキル": "日本語（ネイティブ）、英語（高度なスキル）",\n    "必要なテクニカルスキル": "Python、SQL、MLモデル構築、データ可視化、Webアプリ開発（Dash、Streamlit）"\n}\n',
 [{'thoughts': '\nJob Descriptionから必要な情報を抽出するタスクを理解しました。会社名、ポジション名、必要な学歴、必要な経験年数、必要な言語スキル、必要なテクニカルスキルの6つの要素を特定する必要があります。Job Descriptionには具体的な学歴の情報が含まれていないため、これを「情報なし」として出力します。また、必要な経験年数は8年以上と明記されているため、これを正確に反映します。言語スキルとテクニカルスキルも明確に記載されているため、それに基づいて回答を作成します。\n',
   'result': '\n{\n    "会社名": "Dataiku",\n    "ポジション名": "Senior Data Scientist",\n    "必要な学歴": "情報なし",\n    "必要な経験年数": "8年以上",\n    "必要な言語スキル": "日本語（ネイティブ）、英語（高度なスキル）",\n    "必要なテクニカルスキル": "Python、SQL、MLモデル構築、データ可視化、Webアプリ開発（Dash、Streamlit）"\n}\n'}])

In [18]:
len(reply)

2

In [19]:
reply[0]

'\n{\n    "会社名": "Dataiku",\n    "ポジション名": "Senior Data Scientist",\n    "必要な学歴": "情報なし",\n    "必要な経験年数": "8年以上",\n    "必要な言語スキル": "日本語（ネイティブ）、英語（高度なスキル）",\n    "必要なテクニカルスキル": "Python、SQL、MLモデル構築、データ可視化、Webアプリ開発（Dash、Streamlit）"\n}\n'

In [22]:
print(reply[0])


{
    "会社名": "Dataiku",
    "ポジション名": "Senior Data Scientist",
    "必要な学歴": "情報なし",
    "必要な経験年数": "8年以上",
    "必要な言語スキル": "日本語（ネイティブ）、英語（高度なスキル）",
    "必要なテクニカルスキル": "Python、SQL、MLモデル構築、データ可視化、Webアプリ開発（Dash、Streamlit）"
}



In [23]:
json.loads(reply[0])

{'会社名': 'Dataiku',
 'ポジション名': 'Senior Data Scientist',
 '必要な学歴': '情報なし',
 '必要な経験年数': '8年以上',
 '必要な言語スキル': '日本語（ネイティブ）、英語（高度なスキル）',
 '必要なテクニカルスキル': 'Python、SQL、MLモデル構築、データ可視化、Webアプリ開発（Dash、Streamlit）'}

In [20]:
reply[1]

[{'thoughts': '\nJob Descriptionから必要な情報を抽出するタスクを理解しました。会社名、ポジション名、必要な学歴、必要な経験年数、必要な言語スキル、必要なテクニカルスキルの6つの要素を特定する必要があります。Job Descriptionには具体的な学歴の情報が含まれていないため、これを「情報なし」として出力します。また、必要な経験年数は8年以上と明記されているため、これを正確に反映します。言語スキルとテクニカルスキルも明確に記載されているため、それに基づいて回答を作成します。\n',
  'result': '\n{\n    "会社名": "Dataiku",\n    "ポジション名": "Senior Data Scientist",\n    "必要な学歴": "情報なし",\n    "必要な経験年数": "8年以上",\n    "必要な言語スキル": "日本語（ネイティブ）、英語（高度なスキル）",\n    "必要なテクニカルスキル": "Python、SQL、MLモデル構築、データ可視化、Webアプリ開発（Dash、Streamlit）"\n}\n'}]

In [21]:
len(reply[1])

1